Konu: Website Phishing: Çeşitli web sitelerinden toplanan 9 özniteliğe göre 1353 web 
sitesinin kimlik avı-dolandırıcılık(pishing) tehditlerine göre Meşru:‘1’, Şüpheli: 
‘0’, Dolandırıcı(pishing):’-1’ sınıflandırılmıştır. 
Algoritma:  
LR(LogisticRegression: Lojistik regresyon tabanlı sınıflandırma) 
KNC(KNeighborsClassifier: K-en yakın komşular sınıflandırma) 

# Kütüphanelerin Eklenmesi 

In [230]:
import numpy as np #Bilimsel hesaplama için. Yani diziler, matrisler, matematiksel işlemler diyebiliriz.
import pandas as pd #veri analizi için
import matplotlib.pyplot as plt #görselleştirme için kullanacağız
from sklearn.model_selection import train_test_split #veriyi eğitim ve test verilerine ayırmamızı sağlayacak 
from sklearn.preprocessing import StandardScaler # veri ölçeklendirme yani normalizasyon işlemi için kullanacağız
from sklearn.linear_model import LogisticRegression # LR modelini oluşturmamız için bu kütüphaneyi eklememiz gerek.
#veri performans ölçmek için birden fazla yolu aşağıdaki komut ile ekliyoruz. Ör: f1 scoru gibi 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize # multiclass yapıyı binary formata dönüştürmek için kullanılıyor. 
from itertools import cycle #Döngü oluşturur ve renkler atar. 
from scipy.io import arff #veri dosyasının yapısı yüzünden ilk seferde okuma işlemini gerçekleştiremedim

# Verinin Okunması 

- Veri dosyasını okuyup hakkında bilgi alarak başlayacağız. Ardından bu verileri kontrol edip ona göre hareket edeceğiz. 

In [231]:
#.arff dosyas okuma 
data_arff = arff.loadarff("data/PhishingData.arff")

#veri ile işlem yapabilmeye devam etmem için öncelikle pd ye uyumlu hale getirmeliyim 
data = pd.DataFrame(data_arff)

# Satır ve sütun sayısını kontrol et
print("Satır sayısı:", data.shape[0])
print("Sütun sayısı:", data.shape[1])

print("Veri Setinin Bir Kısmı: ")
print(data.head())

print("\nVeri Seti Hakkında Bilgi:")
print(data.info())

Satır sayısı: 2
Sütun sayısı: 1353
Veri Setinin Bir Kısmı: 
                                                0     ...                                               1352
0  [b'1', b'-1', b'1', b'-1', b'-1', b'1', b'1', ...  ...  [b'1', b'0', b'1', b'1', b'1', b'0', b'-1', b'...
1                                                SFH  ...                                               None

[2 rows x 1353 columns]

Veri Seti Hakkında Bilgi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Columns: 1353 entries, 0 to 1352
dtypes: object(1353)
memory usage: 21.3+ KB
None


## Eksik Değer Kontrolü

In [232]:
print("\n Eksik Değer Kontrolü Yapalım:")
print(data.isnull().sum())

# Tamamen NaN olan sütunları kontrol et
nan_columns = data.columns[data.isnull().all()]
print("\nTamamen NaN olan sütunlar:", nan_columns) 


 Eksik Değer Kontrolü Yapalım:
0       0
1       0
2       0
3       0
4       0
       ..
1348    1
1349    1
1350    1
1351    1
1352    1
Length: 1353, dtype: int64

Tamamen NaN olan sütunlar: Index([], dtype='int64')


- Çıktıya göre boş sütun yok o yüzden kaldırmamız gereken bir sütunda yok demektir. Olsaydı duruma el atmamız lazımdı şekerim <3 

* Çıktı yorumlarsak veri setindeki veriler object türünde. Eksik verileri tamamlamak istediğimizde veri türlerini değiştirmemiz gerekecek yoksa bu kısım ileride hata almamıza sebep olacak. 
- Eksik değer kontrolünde ilk satırlarda eksik değer gözükmüyor fakat sonraki 1348. satırdan itibaren satırlarda eksik görünmeye başlanmış ve bu 1 ile ifade edilmiş. Eksik satırlar doldurulmalı ki model düşük performans ile çalışmasın. 

## Veri Sayısallaştırma

In [233]:
# Object Türündeki sütunları listeleyip bir diziye atayacağım. Ardından dönüşüm yapacağım
non_numeric_columns = data.select_dtypes(include=['object']).columns

# Object türündeki sütunlarda temizleme işlemi
for col in non_numeric_columns:
    data[col] = data[col].str.strip()  # Başındaki ve sonundaki boşlukları temizle
    
# Sayısallaştırma işlemi
data = data.apply(pd.to_numeric, errors='coerce')

print("\nVeri Türleri:")
print(data.dtypes)


Veri Türleri:
0       float64
1       float64
2       float64
3       float64
4       float64
         ...   
1348    float64
1349    float64
1350    float64
1351    float64
1352    float64
Length: 1353, dtype: object


## Eksik Veri Tamamlama 

In [234]:
#eksik değerleri 0 ile dolduralım. Çünkü ne suçlu ne meşru diyebiliriz. 0 şüpheli iyidir :)
# ilk denemem de fillna('0') olarak yazdığım için sayısallaştırma işlemi yine tersine döndü :D 
data = data.fillna(0) 

#sonrasında bir kez daha değerleri kontrol edelim 
print("\n Doldurma İşleminden Sonra Eksik Değer Kontrolü:")
print(data.isnull().sum())



 Doldurma İşleminden Sonra Eksik Değer Kontrolü:
0       0
1       0
2       0
3       0
4       0
       ..
1348    0
1349    0
1350    0
1351    0
1352    0
Length: 1353, dtype: int64


# Önişlem Yapılması

In [235]:
x = data.iloc[:, :-1] #son sütun hariç tüm sütunlar
y = data.iloc[:, -1] #son sütun 

print("Veri seti boyutu:", data.shape)
print("\n x:", x.shape)
print("\n y", y.shape)

Veri seti boyutu: (2, 1353)

 x: (2, 1352)

 y (2,)


# Eğitim ve Test Verilerine Ayrılması 

In [236]:
#veriyi eğitim ve test verileri olmak üzere iki kısma böleceğiz. 
#genelde bunun için 0.2 veya 0.3 kullanılır
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#Şimdi verileri kontrol edelim
print("\nEğitim Verileri:", x_train.shape, x_test.shape)
print("\nTest Verileri:", y_train.shape, y_test.shape)


Eğitim Verileri: (1, 1352) (1, 1352)

Test Verileri: (1,) (1,)


# Veri Ölçeklendirme 

- Normalizasyon işlemi yapacağız

In [237]:
scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

# LR Modeli Oluşturma

- Bu modeli kullanmak için bir kütüphane eklemiştik zaten. Bunu bize scikit-learn kütüphenesi sağladı. O yüzden biz ondan faydalanacağız sadece.

In [ ]:
LR_model = LogisticRegression()
LR_model.fit(x_train_scaled, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.float64(0.0)